In [1]:
import os
import json
import numpy as np

In [2]:
try:
    os.mkdir('afl_stats')
except:
    pass

In [3]:
def save_json(data, filename, _dir='afl_stats'):
    try: # make directory if it doesnt exist
        os.mkdir(_dir)
    except:
        pass
    with open(os.path.join(_dir,filename + '.json'), 'w') as fp:
        json.dump(data, fp)
        
def load_json(filename, _dir='afl_stats'):
#     try: # make directory if it doesnt exist
    with open(os.path.join(_dir,filename), 'r') as fp:
        return json.load(fp)
#     except:
#         raise ValueError()



In [4]:
# from http://stackoverflow.com/questions/23561759/cannot-fetch-javascript-element

import requests
# from bs4 import BeautifulSoup

# page_url = 'http://www.afl.com.au/afl/stats/player-ratings/overall-standings'
token_url = 'http://www.afl.com.au/api/cfs/afl/WMCTok'
# player_url = 'http://www.afl.com.au/api/cfs/afl/playerProfile/'

session = requests.Session()
# r = session.get(page_url)
# soup = BeautifulSoup(r.content)
token = session.post(token_url).json()['token']


In [5]:
data = session.post(token_url)

{u'disclaimer': u'All content and material contained within this site is protected by copyright owned by or licensed to Telstra. Unauthorised reproduction, publishing, transmission, distribution, copying or other use is prohibited.',
 u'token': u'82ad35bbe2a0068fb85cf95364136d4c'}

In [6]:
token

u'fd19c3aece068484f199526ee8345bae'

FROM https://github.com/samvrlewis/afl-stats-database/blob/master/aflcomau_scraper_stub.py

Some code to get the JSON from the AFL.com.au stats api
I found the header info by using the developer tools in Chrome:
	- Visit http://www.afl.com.au/stats
	- Open developer tools
	- Change to network tab
	- Add a filter for 'XHR'
	- Change season to 2014, round to round 1 and match to collingwood vs freo
	- There should be a GET request in the developer tools log 
	- Can examine the request to see the request headers, cookies etc
It only seems to work if the X-media-mis-token header is included, I'm not sure how this is generated
I'm also not sure how the query string is generated but this could probably be found by looking at a few different requests
and looking for the pattern
competitionId I would guess is constant
roundId and matchId might just be the date + game number or something?

In [7]:
import pandas as pd

In [8]:
# GET /api/cfs/afl/statsCentre/players?competitionId=CD_S2014014&roundId=CD_R201401401 HTTP/1.1
headers = {
    "Host": "www.afl.com.au",
    "Connection": "keep-alive",
    "Accept": "application/json, text/javascript, */*; q=0.01",
    "X-media-mis-token": token,
    "X-Requested-With": "XMLHttpRequest",
    "User-Agent": r'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.124 Safari/537.36',
    "Referer": "http://www.afl.com.au/stats",
    "Accept-Encoding": "gzip, deflate, sdch",
    "Accept-Language": "en-GB,en-US;q=0.8,en;q=0.6",
    }

In [9]:
query_string = 'http://www.afl.com.au/api/cfs/afl/matchItems/round/CD_R200101412'
response = session.get(query_string, headers=headers)
match_json = response.json()
match_json

{u'byes': [],
 u'items': [{u'liveVideos': [],
   u'match': {u'abbr': u'ESS V ADEL',
    u'awayTeam': {u'abbr': u'ADEL',
     u'badgeAssetURL': None,
     u'bio': None,
     u'coachId': None,
     u'flagAssetURL': None,
     u'guid': None,
     u'name': u'Adelaide Crows',
     u'nickname': u'Crows',
     u'photoURL': None,
     u'players': None,
     u'teamId': u'CD_T10',
     u'timeZone': None},
    u'awayTeamGuid': u'dd5bbb12fafb7310VgnVCM100000976fb70aRCRD',
    u'awayTeamId': u'CD_T10',
    u'awayTeamPostGameArticleURL': None,
    u'awayTeamPreGameArticleURL': None,
    u'chatId': None,
    u'date': u'2001-06-15T04:10:00.000+0000',
    u'guid': u'932abba74bdf7310VgnVCM100000976fb70aRCRD',
    u'hash': u'5aca9bee04294c068427bf0acaf3a72b',
    u'homeTeam': {u'abbr': u'ESS',
     u'badgeAssetURL': None,
     u'bio': None,
     u'coachId': None,
     u'flagAssetURL': None,
     u'guid': None,
     u'name': u'Essendon',
     u'nickname': u'Bombers',
     u'photoURL': None,
     u'players

In [10]:
def get_round(_round, year):
    def q_string(_round, year):
        return 'http://www.afl.com.au/api/cfs/afl/statsCentre/players?competitionId=CD_S{}014&roundId=CD_R{}014{:0>2d}'.format(year, year, _round)

    query_string = q_string(_round, year)
#     print query_string
    response = session.get(query_string, headers=headers)
    match_json = response.json()
    return match_json

In [11]:
player_keys = [
#     u'timeOnGroundPercentage',
                 u'surname',
#                  u'photoURL',
                 u'playerId',
#                  u'gamesPlayed',
                 u'givenName',
                 u'jumper']

def format_output(raw, _round, year):
    outlist = []
    for i in raw['lists']:
        d = {x: i['player'].get(x, np.NaN) for x in player_keys}
        d.update(i['stats']['totals'])
        d.update(i['team'])
        d.update({'year':year, 'afl_round':_round})
        outlist.append(d)
    return outlist

In [12]:
# res = []
# for year in range(2001,2016):
#     for i in range(1,30):
#         roundId = 'CD_R{}014{:0>2d}'.format(year, i)
# #         print roundId
#         round_file = 'afl_stats/rounds/'+roundId+'.json'
# #         if os.path.isfile(round_file):
# #             print '.',
# #             sample = format_output(load_json(round_file), i, year)
# #             res.extend(sample)
# # #             print sample
# # #             break
# #         else:
#         print 'Getting R{}-{}'.format(i, year)
#         raw = get_round(i,year)
#         try:
#             sample = format_output(raw, i, year)
#         except Exception as e: 
#             sample = []
#             print 'Cant get R{}-{}'.format(i, year)
# #                 print e
#         res.extend(sample)

In [13]:
def get_matches(_round, year):
    # http://www.afl.com.au/api/cfs/afl/matchItems/round/CD_R201501406
    
    roundId = 'CD_R{}014{:0>2d}'.format(year, _round)
    query_string = 'http://www.afl.com.au/api/cfs/afl/matchItems/round/'+roundId

    response = session.get(query_string, headers=headers)
    match_json = response.json()
    return match_json, roundId


In [14]:
data, roundid = get_matches(5,2015)

In [15]:
# Code to get all round information and build list of matches
# Saves information for every round as json

In [16]:
match_dict = {}
for year in range(2001,2016):
    print '\n'+str(year),
    for i in range(1,30):
        roundid = 'CD_R{}014{:0>2d}'.format(year, i)
        if os.path.isfile('afl_stats/rounds/'+roundid+'.json'):
            print '.',
            raw = load_json(roundid+'.json', _dir='afl_stats/rounds/')
            matches = [item['match']['matchId'] for item in raw['items']] # Get list of matchids for the round
            match_dict[roundid] = matches
        
        else:
            raw, roundid = get_matches(i,year)
#             print '\nGetting results for round {} {}, roundID: {}'.format(i, year, roundid)

            try:
                matches = [item['match']['matchId'] for item in raw['items']] # Get list of matchids for the round
                save_json(raw, roundid, _dir='afl_stats/rounds/')
                match_dict[roundid] = matches
                print '+',
            except:
#                 print 'Error with results for round {} {}'.format(i, year)
                print 'X',
                save_json(raw, 'error_'+roundid, _dir='afl_stats/rounds/')


2001 . . . . . . . . . . . . . . . . . . . . . . . . . . X X X 
2002 . . . . . . . . . . . . . . . . . . . . . . . . . . X X X 
2003 . . . . . . . . . . . . . . . . . . . . . . . . . . X X X 
2004 . . . . . . . . . . . . . . . . . . . . . . . . . . X X X 
2005 . . . . . . . . . . . . . . . . . . . . . . . . . . X X X 
2006 . . . . . . . . . . . . . . . . . . . . . . . . . . X X X 
2007 . . . . . . . . . . . . . . . . . . . . . . . . . . X X X 
2008 . . . . . . . . . . . . . . . . . . . . . . . . . . X X X 
2009 . . . . . . . . . . . . . . . . . . . . . . . . . . X X X 
2010 . . . . . . . . . . . . . . . . . . . . . . . . . . . X X 
2011 . . . . . . . . . . . . . . . . . . . . . . . . . . . . X 
2012 . . . . . . . . . . . . . . . . . . . . . . . . . . . X X 
2013 . . . . . . . . . . . . . . . . . . . . . . . . . . . X X 
2014 . . . . . . . . . . . . . . . . . . . . . . . . . . . X X 
2015 . . . . . . . . . . . . . . . . . . . . . . . . + + + X X


In [17]:
match_list = []
for k,v in iter(sorted(match_dict.iteritems())) :
    match_list.extend(v)

In [18]:
with open('afl_stats/match_list.json', 'wb') as fp:
    # Make dictionary full of data
    json.dump(match_list, fp)

In [19]:
def stats_query(matchId):
    query_string = 'http://www.afl.com.au/api/cfs/afl/statsCentre/players?matchId='+matchId
    response = session.get(query_string, headers=headers)
    match_json = response.json()
    return match_json

In [20]:
playerIds = set()

In [21]:
def retry_endlessly(func, *args):
    try:
        return func(*args)
    except:
        return retry_endlessly(func, *args)
    

In [22]:
with open('afl_stats/match_list.json', 'r') as fp:
    # Make dictionary full of data
    match_list = json.load(fp)

In [23]:
# year = [4:8]
# round =  i[-4:-2]
#[i[-4:-2] for i in match_list[:50]]

In [24]:
player_stats = []
for n, matchId in enumerate(match_list):
    if os.path.isfile('afl_stats/match_player_stats/'+matchId+'.json'):
#         print '{} already downloaded'.format(matchId)
        print '.',
    else:
        print 'Getting player stats for match {}: {} of {}'.format(matchId, n, len(match_list))
        raw = retry_endlessly(stats_query, matchId)

        try:
            players = [pl['player']['playerId'] for pl in raw['lists']]
            for i in players:
                playerIds.add(i) # Build up list of players in database
            save_json(raw, matchId, _dir='afl_stats/match_player_stats/')

        except Exception:
            print 'Error with results for match {}'.format(matchId)
            save_json(raw, 'error_'+matchId, _dir='afl_stats/match_player_stats/')
    try:
        data = load_json(matchId+'.json', _dir='afl_stats/match_player_stats/')
        data = format_output(data, int(matchId[-4:-2]), int(matchId[4:8]))
        player_stats.extend(data)
    except:
        pass
        #     player_stats.e

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

In [25]:
player_df = pd.DataFrame(player_stats)

In [26]:
player_df.to_csv('player_stats_df.csv')

In [27]:
playerlist = player_df.playerId.unique()

In [28]:
def getFromDict(dataDict, mapList):
    #http://stackoverflow.com/questions/14692690/access-python-nested-dictionary-items-via-a-list-of-keys
    return reduce(lambda d, k: d[k], mapList, dataDict)

In [29]:
keylist = {
    'awayteam': ['match','awayTeam','abbr'],
    'awayteam_id': ['match','awayTeam','teamId'],
    'date': ['match','date'],
    'hometeam': ['match','homeTeam','abbr'],
    'hometeam_id': ['match','homeTeam','teamId'],
    'match_id': ['match','matchId'],
    'round_id': ['match','round'],
    'year': ['round','year'],
    'roundNumber': ['round','roundNumber'],
    'venue': ['venue','abbreviation'],
    'venue_name': ['venue','name'],
    'venue_id': ['venue','venueId'],
    }

In [30]:
for k,v in keylist.iteritems():
    print k.ljust(16), getFromDict(test['items'][0], v)

match_id        

NameError: name 'test' is not defined

In [31]:
def get_player(playerId):
    # http://www.afl.com.au/api/cfs/afl/matchItems/round/CD_R201501406
    query_string = 'http://www.afl.com.au/api/cfs/afl/playerProfile/'+playerId

    response = session.get(query_string, headers=headers)
    r = response.json()
    save_json(r, playerId,'afl_stats/player_summary')
    

In [32]:
# with open('afl_stats/playerId_list.json', 'r') as fp:
#     # Make dictionary full of data
#     playerlist = json.load(fp)

In [34]:
for n, playerId in enumerate(playerlist):
#     if os.path.isfile('afl_stats/player_summary/'+playerId+'.json'):
#         print '{} already downloaded'.format(playerId)
#         print '.',
#     else:
    print 'Getting player stats for player {}: {} of {}'.format(playerId, n, len(playerlist))
#         raw = retry_endlessly(stats_query, matchId)

    try:
        get_player(playerId)

    except Exception:
        print 'Error with results for player {}'.format(playerId)
#             save_json(raw, 'error_'+matchId, _dir='afl_stats/match_player_stats/')

Getting player stats for player CD_I960309: 0 of 1817
Getting player stats for player CD_I960473: 1 of 1817
Getting player stats for player CD_I960474: 2 of 1817
Getting player stats for player CD_I960318: 3 of 1817
Getting player stats for player CD_I960250: 4 of 1817
Getting player stats for player CD_I960504: 5 of 1817
Getting player stats for player CD_I960338: 6 of 1817
Getting player stats for player CD_I960597: 7 of 1817
Getting player stats for player CD_I210014: 8 of 1817
Getting player stats for player CD_I200049: 9 of 1817
Getting player stats for player CD_I960008: 10 of 1817
Getting player stats for player CD_I960493: 11 of 1817
Getting player stats for player CD_I990048: 12 of 1817
Getting player stats for player CD_I210023: 13 of 1817
Getting player stats for player CD_I970007: 14 of 1817
Getting player stats for player CD_I990055: 15 of 1817
Getting player stats for player CD_I960160: 16 of 1817
Getting player stats for player CD_I960499: 17 of 1817
Getting player stats